# Battle of the Neighborhoods: - Atlanta Venues vs Crime

## Introduction

For the last couple of labs and home work sessions in the class, we have had chance to analyze New York City, San Francisco, and Toronto data. I am working with my sister to finalize her move to Atlanta, US. And during her move to the US, we are mainly interested in find a nice and safe neighborhood for her to stay at.

Thus, in this project, we will analyze venues and crime data  Atlanta. We will try to see if we can find any correlation between the types of venues in a neighborhood and the number of crimes or types of crimes in that neighborhood. Another question we will try to answer would be if clusters of neighborhoods with similar venue types will have similar quantity and/or type of crime. We may also look in to venue ratings once clusters are made to explore whether a lower average rating of venues has any correlation to crime.

The use of this analysis can be expand to anyone who is interested in the relation between venue types and crime of a location. Possible user for the analysis can be the police department, business/venue owners, or policy maker. 

## Data Description

#### Neighbor Planning Units

We will use Wikipedia to get data for Neighborhood Planning Units (NPUs). NPUs are neighborhood-scale governmental structure used in the city of Atlanta. There are 25 NPUs in Atlant, lettered from A-Z (except U). This data will be used to determind the coordinate for each neighborhood in Atlanta.

The data can be accessed from:
https://en.wikipedia.org/wiki/Neighborhood_planning_unit

Example of the data can be seen below:

In [1]:
import requests
import pandas as pd

In [2]:
npu = pd.read_csv('npu.csv')
npu.head()

,NPU,Neighborhood
0,A,Chastain Park
1,A,Kingswood
2,A,Margaret Mitchell
3,A,Mt. Paran Parkway
4,A,Mt. Paran/Northside


#### Atalanta Crime Data

The Atlanta PD provides listing of all type of crimes by NPU and Neighborhood, along with its coordinate. The coordinate will be used to merge with Foursquare API data to determine the nearby venues associated with a crime. The data provided by Atlanta PD spans from 2009 to 2020. For simplicity and completeness of data, we will only use data for full year 2019.

The data can be downloaded from: https://www.atlantapd.org/i-want-to/crime-data-downloads

Example of the data can be seen below:

In [3]:
crime = pd.read_csv('COBRA-2009-2019.csv')
crime.head()

C:\Users\halol\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Report Number,Report Date,Occur Date,Occur Time,Possible Date,Possible Time,Beat,Apartment Office Prefix,Apartment Number,Location,Shift Occurence,Location Type,UCR Literal,UCR #,IBR Code,Neighborhood,NPU,Latitude,Longitude
0,90010930,2009-01-01,2009-01-01,1145,2009-01-01,1148.0,411.0,NaN,NaN,2841 GREENBRIAR PKWY,Day Watch,8,LARCENY-NON VEHICLE,630,2303,Greenbriar,R,33.68845,-84.49328
1,90011083,2009-01-01,2009-01-01,1330,2009-01-01,1330.0,511.0,NaN,NaN,12 BROAD ST SW,Day Watch,9,LARCENY-NON VEHICLE,630,2303,Downtown,M,33.75320,-84.39201
2,90011208,2009-01-01,2009-01-01,1500,2009-01-01,1520.0,407.0,NaN,NaN,3500 MARTIN L KING JR DR SW,Unknown,8,LARCENY-NON VEHICLE,630,2303,Adamsville,H,33.75735,-84.50282
3,90011218,2009-01-01,2009-01-01,1450,2009-01-01,1510.0,210.0,NaN,NaN,3393 PEACHTREE RD NE,Evening Watch,8,LARCENY-NON VEHICLE,630,2303,Lenox,B,33.84676,-84.36212
4,90011289,2009-01-01,2009-01-01,1600,2009-01-01,1700.0,411.0,NaN,NaN,2841 GREENBRIAR PKWY SW,Unknown,8,LARCENY-NON VEHICLE,630,2303,Greenbriar,R,33.68677,-84.49773


#### Foursquare API 

Like in other labs, we will use Foursquare API to get venue information.

Example of the data can be seen below

In [4]:
example_df = crime[['Neighborhood','Latitude','Longitude']].loc[0:3,]
example_df

,Neighborhood,Latitude,Longitude
0,Greenbriar,33.68845,-84.49328
1,Downtown,33.75320,-84.39201
2,Adamsville,33.75735,-84.50282
3,Lenox,33.84676,-84.36212


In [5]:
CLIENT_ID = 'HFL0RQARIH2YPHFZFHQWPDIHWJF4XC3BBY3PWEELDFDAXTZL' # your Foursquare ID
CLIENT_SECRET ='ARGKXWTQXFUJHASLN0NDVD4G2KOWC2L03MOO01G4SHW3QW0D' # your Foursquare Secret
VERSION = '20200101' # Foursquare API version

LIMIT = 2 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

example = getNearbyVenues(names=example_df['Neighborhood'],
                                   latitudes=example_df['Latitude'],
                                   longitudes=example_df['Longitude']
                                  )

example

Greenbriar
Downtown
Adamsville
Lenox


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Greenbriar,33.68845,-84.49328,Planet Fitness,33.687053,-84.491564,Gym / Fitness Center
1,Greenbriar,33.68845,-84.49328,Piccadilly Cafeteria,33.688738,-84.494511,Southern / Soul Food Restaurant
2,Downtown,33.75320,-84.39201,Jamrock Restaurant,33.751554,-84.391356,Caribbean Restaurant
3,Downtown,33.75320,-84.39201,Walgreens,33.754345,-84.389484,Pharmacy
4,Adamsville,33.75735,-84.50282,Planet Fitness,33.759286,-84.504357,Gym / Fitness Center
5,Adamsville,33.75735,-84.50282,Dollar General,33.757484,-84.502062,Discount Store
6,Lenox,33.84676,-84.36212,Forum Athletic Club,33.846695,-84.361770,Gym
7,Lenox,33.84676,-84.36212,True Food Kitchen,33.847687,-84.363843,New American Restaurant
